In [1]:
%cd /home/yz979/code/kaggle-perturbation
!ls data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scanpy as sc

/storage/home/yz979/code/kaggle-perturbation
adata_obs_meta.csv   de_train.h5ad	     sample_submission.csv
adata_train.h5ad     de_train.parquet	     submission_1.csv
adata_train.parquet  id_map.csv		     submission_2.csv
cpa_model	     id_map.h5ad	     submission_3.csv
cpa_model_2	     multiome_obs_meta.csv   unzip.py
cpa_model_adata      multiome_train.parquet
de_train_genes.txt   multiome_var_meta.csv


## Explore ``adata_train.parquet``

In [11]:
adata_train = pd.read_parquet('data/adata_train.parquet')

In [55]:
unmelt = adata_train.pivot_table(index="obs_id", columns="gene", values="normalized_count").fillna(0)

/tmp/ipykernel_263081/2280732669.py:1: PerformanceWarning: The following operation may generate 5103112950 cells in the resulting pandas object.
  unmelt = adata_train.pivot_table(index="obs_id", columns="gene", values="normalized_count").fillna(0)


In [56]:
adata_obs_meta = pd.read_csv('data/adata_obs_meta.csv', index_col=0)

In [59]:
adata = anndata.AnnData(X=unmelt.values, obs=adata_obs_meta)
adata.write_h5ad('data/adata_train.h5ad')

In [60]:
adata

AnnData object with n_obs × n_vars = 240090 × 21255
    obs: 'library_id', 'plate_name', 'well', 'row', 'col', 'cell_id', 'donor_id', 'cell_type', 'sm_lincs_id', 'sm_name', 'SMILES', 'dose_uM', 'timepoint_hr', 'control'

In [34]:
adata_obs_meta = pd.read_csv('data/adata_obs_meta.csv')
len(adata_obs_meta['sm_name'].unique().tolist())

147

In [50]:
adata_obs_meta.head()

,obs_id,library_id,plate_name,well,row,col,cell_id,donor_id,cell_type,sm_lincs_id,sm_name,SMILES,dose_uM,timepoint_hr,control
0,000006a87ba75b72,library_4,plate_4,F7,F,7,PBMC,donor_2,T cells CD4+,LSM-4944,MLN 2238,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,1.0,24,False
1,0000233976e3cd37,library_0,plate_3,D4,D,4,PBMC,donor_1,T cells CD4+,LSM-46203,BMS-265246,CCCCOc1c(C(=O)c2c(F)cc(C)cc2F)cnc2[nH]ncc12,1.0,24,False
2,0001533c5e876362,library_2,plate_0,B11,B,11,PBMC,donor_0,T regulatory cells,LSM-45663,Resminostat,CN(C)Cc1ccc(S(=O)(=O)n2ccc(/C=C/C(=O)NO)c2)cc1,1.0,24,False
3,00022f989630d14b,library_35,plate_2,E6,E,6,PBMC,donor_0,T cells CD4+,LSM-43216,FK 866,O=C(/C=C/c1cccnc1)NCCCCC1CCN(C(=O)c2ccccc2)CC1,1.0,24,False
4,0002560bd38ce03e,library_22,plate_4,B6,B,6,PBMC,donor_2,T cells CD4+,LSM-1099,Nilotinib,Cc1cn(-c2cc(NC(=O)c3ccc(C)c(Nc4nccc(-c5cccnc5)...,1.0,24,False


In [3]:
de_train = pd.read_parquet('data/de_train.parquet')
X = de_train.iloc[:, 5:]

# de_train_ = anndata.AnnData(X=X, obs=de_train.iloc[:, :5])

# # save adata
# de_train_.write('data/de_train.h5ad')

In [ ]:
genes = de_train_.var_names

In [35]:
len(de_train['sm_name'].unique().tolist())

146

In [2]:
de_train

# find 1:1 mapping of sm_name and sm_lincs_id

sm_name = de_train.sm_name.unique()
sm_lincs_id = de_train.sm_lincs_id.unique()


def find_sm_lincs_id(sm_name):
    return de_train[de_train.sm_name == sm_name].sm_lincs_id.unique()[0]


sm_lincs_id_map = {sm_name: find_sm_lincs_id(sm_name) for sm_name in sm_name}

NameError: name 'de_train' is not defined

In [49]:
multiome_train = pd.read_parquet('data/multiome_train.parquet')

In [39]:
multiome_train.head()

(216251368, 4)

In [40]:
multiome_var_meta = pd.read_csv('data/multiome_var_meta.csv')

In [43]:
multiome_var_meta

,location,gene_id,feature_type,genome,interval
0,A1BG,ENSG00000121410,Gene Expression,GRCh38,chr19:58353491-58353492
1,A1BG-AS1,ENSG00000268895,Gene Expression,GRCh38,chr19:58347750-58351970
2,A2M,ENSG00000175899,Gene Expression,GRCh38,chr12:9116156-9116157
3,A2M-AS1,ENSG00000245105,Gene Expression,GRCh38,chr12:9065162-9065177
4,A2ML1,ENSG00000166535,Gene Expression,GRCh38,chr12:8822620-8845004
...,...,...,...,...,...
158200,chrY:7765105-7765991,chrY:7765105-7765991,Peaks,GRCh38,chrY:7765105-7765991
158201,chrY:7814158-7815060,chrY:7814158-7815060,Peaks,GRCh38,chrY:7814158-7815060
158202,chrY:7818681-7819599,chrY:7818681-7819599,Peaks,GRCh38,chrY:7818681-7819599
158203,chrY:8535565-8536421,chrY:8535565-8536421,Peaks,GRCh38,chrY:8535565-8536421


In [4]:
id_map = pd.read_csv('data/id_map.csv', index_col='id')
# id_map to parquet
id_map.to_parquet('data/id_map.parquet')

# id_map to anndata
X_ = np.zeros((id_map.shape[0], X.shape[1]))
X_ = pd.DataFrame(X_, index=id_map.index, columns=X.columns)

# false
id_map['control'] = np.zeros(id_map.shape[0], dtype=bool)

# all row of X_ is the average of all rows in X that has the same cell type
for i, row in id_map.iterrows():
    cell_type = row['cell_type']
    X_.loc[i] = X[de_train['cell_type'] == cell_type].mean()

adata_ = anndata.AnnData(X=X_, obs=id_map)
adata_.write('data/id_map.h5ad')

/home/yz979/miniconda3/envs/kaggle/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
submission_1 = pd.read_csv('data/submission_1.csv')
submission_1

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.197018,0.366643,0.308561,0.175244,0.373812,0.920601,-0.239672,0.831461,-0.014554,...,0.211342,0.375856,-0.100039,0.982504,0.220455,0.005646,0.389817,0.547818,-0.258544,-0.373556
1,1,0.006058,0.261375,0.704996,-0.259771,0.759611,1.005394,0.063424,0.581806,-0.046093,...,0.720613,0.453576,-0.374567,0.631949,-0.119916,0.104561,0.257771,0.576432,0.088900,-0.419222
2,2,0.160660,0.574869,0.354372,0.253214,0.866350,1.202243,-0.055843,0.901264,0.078745,...,0.084664,0.347534,-0.265437,0.784556,-0.164767,0.396333,0.697948,0.415502,-0.230277,0.019580
3,3,0.287419,0.494772,0.477845,-0.151231,0.742311,0.786783,-0.078950,0.679512,-0.065502,...,0.814934,-0.235784,-0.272828,0.757581,0.162597,0.137842,0.803988,0.283533,-0.157342,-0.173607
4,4,0.067297,0.301806,0.307276,0.017520,0.435710,0.932309,-0.331595,0.377311,0.093583,...,0.020295,-0.248578,-0.105558,0.217618,-0.030106,-0.133885,0.114168,0.177040,-0.092886,-0.424144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,-0.177983,0.010840,-0.561487,0.094067,0.726233,0.981905,-0.463734,0.527508,0.082562,...,0.281938,-0.214532,-0.096560,0.362515,0.410678,0.049686,0.794873,0.303209,-0.308089,-0.255348
251,251,0.008460,0.640407,-0.384475,0.041072,0.613357,1.143447,-0.160987,0.787393,-0.152795,...,0.050901,-0.111304,-0.249298,0.515100,0.224220,0.088592,0.698528,-0.114826,-0.280496,-0.135796
252,252,0.221061,0.474568,-0.268589,-0.187648,0.861149,0.720751,-0.333910,0.484410,0.146414,...,-0.301899,0.084221,0.168221,0.512570,0.381709,0.449777,1.122832,0.246775,0.007591,-0.396732
253,253,0.561070,0.445395,0.001177,-0.406358,0.683171,1.301565,-0.127820,0.517764,0.169733,...,-0.332796,-0.321881,0.126019,0.569563,0.312362,0.519565,0.992909,-0.237435,-0.074244,0.339248


In [52]:
!kaggle competitions submit -c open-problems-single-cell-perturbations -f data/submission_3.csv -m "submit"

100%|██████████████████████████████████████| 84.8M/84.8M [00:01<00:00, 45.1MB/s]
Successfully submitted to Open Problems – Single-Cell Perturbations